In [1]:
import sys
sys.path.insert(0, 'D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Classes')

import warnings
warnings.filterwarnings('ignore')

from models import EmbeddingsModel
from trainer import EmbeddingTrainer, all_subjects, AccuracyTester, all_subjects_intense

from tqdm import tqdm

import csv
import pandas as pd

In [30]:
use_model = False

saving_path = "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\PainClassificationTest\Results\\xite_phasic_heat.csv"

In [36]:
already_processed = list(pd.read_csv(saving_path)["subject"])
not_processed = [sub for sub in all_subjects if sub not in already_processed]

In [ ]:
for subj in tqdm(not_processed):

    subjects_test = [subj]
    subjects_train = [sub for sub in all_subjects if sub not in subjects_test]


    trainer_hyperparameter = {
        "path_train": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects_no_outliner.pkl",
        "subjects_train": subjects_train,
        "path_test": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects_no_outliner.pkl",
        "subjects_test": subjects_test,
        "acc_tester_metric": "AMI",
        "wandb": False,
        "acc_in_loop": False,
        "learning_rate": 0.001,
        "batch_size": 256,
        "margin": 0.45688,
        "distance": 1,
        "filter": [1, 2, 3]
    }

    model_hyperparameter = {
        "sigmoid": False,
        "dropout": 0.35,
        "layers": [207, 512, 64]
    }


    filter = lambda data: data["label"].isin(trainer_hyperparameter["filter"])

    #model if needed
    if use_model:
        model = EmbeddingsModel(model_hyperparameter)
        trainer = EmbeddingTrainer(trainer_hyperparameter, model, filter=filter, device="cuda:0")
        trainer.trainloop(5)


    #acc testet
    acc_tester_hyperparameter = {
        "path": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects_no_outliner.pkl",
        "subjects_train": subjects_train,
        "subjects_test": subjects_test
    }
    acc_tester = AccuracyTester(acc_tester_hyperparameter, filter=filter, device="cuda:0")

    if use_model:
        acc = acc_tester.test_model(model=model)
    else:
        acc = acc_tester.test_model(max_depth=16)

    with open(saving_path, mode='a') as result_file:
        results_writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        results_writer.writerow([subj, acc*100])

In [31]:
#init file
#with open(saving_path, mode='w') as result_file:
#    results_writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#    results_writer.writerow(["subject", "accuracy"])